In [266]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.renderers.default = 'colab' # использую колаб 
pio.templates.default = 'plotly'

In [267]:
aapl_df = pd.read_csv('AAPL.csv')
btc_df = pd.read_csv('BTC.csv')
gas_df = pd.read_csv('GAS.csv')
aapl_df['date'] = aapl_df['date'].apply(lambda x: pd.to_datetime(x))
btc_df['date'] = btc_df['date'].apply(lambda x: pd.to_datetime(x))
gas_df['Date'] = gas_df['Date'].apply(lambda x: pd.to_datetime(x))

In [268]:
btc_df

,date,Open,High,Low,Close,Volume
0,2019-09-23,10028.05,10049.99,9615.77,9702.25,31937.232356
1,2019-09-24,9702.20,9794.99,7800.00,8493.14,94007.345203
2,2019-09-25,8497.55,8730.00,8215.64,8430.05,60783.892258
3,2019-09-26,8430.05,8465.99,7750.00,8063.73,67930.853749
4,2019-09-27,8063.49,8265.00,7852.15,8177.91,43882.924625
...,...,...,...,...,...,...
1254,2023-02-28,23492.09,23600.00,23020.97,23141.57,264140.998940
1255,2023-03-01,23141.57,24000.00,23020.03,23628.97,315287.417370
1256,2023-03-02,23629.76,23796.93,23195.90,23465.32,239315.452190
1257,2023-03-03,23465.32,23476.95,21971.13,22354.34,319954.197850


In [269]:
fig = px.scatter(data_frame=aapl_df, x='date', y='Adj Close', trendline='ols', title='AAPL')
fig.show()


In [270]:
fig = px.scatter(data_frame=btc_df, x='date', y='Close', trendline='ols', title='BTC')
fig.show()


In [271]:
fig = px.scatter(data_frame=gas_df, x='Date', y='Close', trendline='ols', title='GAS(N.NG)')
fig.show()


In [272]:
# на основе Close извлеките логарифмическую доходность BTC task1, ответ – мю и сигма
# проверьте распределение получившихся данных BTC task2, ответ yes/no 
# возмонжо попробуйте их (BTC) нормализовать task3, ответ – mu и sigma
# попробуйте предположить по подвыборке размером 634 дней среднее значение для доходностей (BTC) на уровне 90 95 99 – task 4, ответ – val + [a;b]
# проверьте корреляцию доходностей BTC & AAPL – task5, ответ – yes/no
# проверьте гипотезу о равенствен средних и дисперсии доходностей BTC & AAPL & GAS – task6, ответ – yes/no

1) На основе Close извлеките логарифмическую доходность BTC task1, ответ – мю и сигма

In [273]:
import numpy as np

# Logarithmic Return Rate (логарифмическая доходность)
btc_lrr = np.log(btc_df['Close']/btc_df['Close'].shift(1)).dropna() 
btc_lrr_mean = np.mean(btc_lrr) 
btc_lrr_std = np.std(btc_lrr)
print(f'Мю - {btc_lrr_mean}, Сигма {btc_lrr_std}')

Мю - 0.000663301751786781, Сигма 0.038671397386446926


2) Проверьте распределение получившихся данных BTC task2, ответ yes/no 

In [274]:
from scipy import stats


data = btc_lrr

# тест Шапиро-Уилка
statistic, p_value = stats.shapiro(data)
alpha = 0.05  
if p_value > alpha:
    print("Тест Шапиро-Уилка - нормальное распределение")
else:
    print("Тест Шапиро-Уилка - не нормальное распределение")

# тест Андерсона-Дарлинга:
result = stats.anderson(data)
critical_values = result.critical_values
significance_levels = result.significance_level
if result.statistic < critical_values[2]:
    print("Тест Андерсона-Дарлинга - нормальное распределение")
else:
    print("Тест Андерсона-Дарлинга - не нормальное распределение")
  
# тест Колмогорова-Смирнова:
statistic, p_value = stats.kstest(data, 'norm')
alpha = 0.05  
if p_value > alpha:
    print("Тест Колмогорова-Смирнова - нормальное распределение")
else:
    print("Тест Колмогорова-Смирнова - не нормальное распределение")

Тест Шапиро-Уилка - не нормальное распределение
Тест Андерсона-Дарлинга - не нормальное распределение
Тест Колмогорова-Смирнова - не нормальное распределение


In [275]:
print("Итого, по результатам трех тестов, ответ - no")

Итого, по результатам трех тестов, ответ - no


3) Возмонжо попробуйте их (BTC) нормализовать task3, ответ – mu и sigma

In [276]:
# придется нормализовывать...
btc_lrr_normalize = ((btc_lrr - btc_lrr_mean) / btc_lrr_std).dropna()
btc_lrr_normalize_mean = np.mean(btc_lrr_normalize)
btc_lrr_normalize_std = np.std(btc_lrr_normalize)
print(f'Мю - {btc_lrr_normalize_mean}, Сигма {btc_lrr_normalize_std}')

Мю - 1.129638689602703e-17, Сигма 1.0


4) Попробуйте предположить по подвыборке размером 634 дней среднее значение для доходностей (BTC) на уровне 90 95 99 – task 4, ответ – val + [a;b]

In [277]:
btc_lrr_634 = btc_lrr.sample(634)
btc_lrr_size = len(btc_lrr)
btc_lrr_634_mean = np.mean(btc_lrr_634)
btc_lrr_634_std = np.std(btc_lrr_634)

# критические значения t-распределения
t_values = stats.t.ppf([0.9, 0.95, 0.99], df=btc_lrr_size-1)
# доверительные интервалы
confidence_intervals = [(btc_lrr_634_mean - t_value * btc_lrr_634_std / np.sqrt(btc_lrr_size), 
                         btc_lrr_634_mean + t_value * btc_lrr_634_std / np.sqrt(btc_lrr_size)) for t_value in t_values]

print("Среднее значение доходности (val) - ", btc_lrr_634_mean)
print("Доверительный интервал [a;b] с уровнем значимости 90%: ", confidence_intervals[0])
print("Доверительный интервал [a;b] с уровнем значимости 95%: ", confidence_intervals[1])
print("Доверительный интервал [a;b] с уровнем значимости 99%: ", confidence_intervals[2])

Среднее значение доходности (val) -  0.0008403451415741908
Доверительный интервал [a;b] с уровнем значимости 90%:  (-0.0006312424522549423, 0.002311932735403324)
Доверительный интервал [a;b] с уровнем значимости 95%:  (-0.0010488167551173547, 0.0027295070382657364)
Доверительный интервал [a;b] с уровнем значимости 99%:  (-0.0018329723735048876, 0.0035136626566532694)


5) Проверьте корреляцию доходностей BTC & AAPL – task5, ответ – yes/no

In [278]:
# предобрабатываем aapl и мержим с btc
aapl_lrr = np.log(aapl_df['Close'] / aapl_df['Close'].shift(1))
data = pd.concat([aapl_lrr.rename('aapl_lrr'), btc_lrr.rename('btc_lrr')], axis=1).dropna()
# считаем корреляцию 
correlation = data['aapl_lrr'].corr(data['btc_lrr'])
print(f'Корреляция - {correlation}')

Корреляция - 0.021809532287096853


In [279]:
# Значение корреляции находится в диапазоне от -1 до 1
# Отрицательное значение указывает на обратную корреляцию, а положительное - на прямую
# Значение близкое к 0 означает отсутствие корреляции
print('В нашем случае корреляция отсутсвует, ответ - no')

В нашем случае корреляция отсутсвует, ответ - no


6) Проверьте гипотезу о равенствен средних и дисперсии доходностей BTC & AAPL & GAS – task6, ответ – yes/no

In [280]:
# снова преобработка и мержинг
gas_lrr = np.log(gas_df['Close'] / gas_df['Close'].shift(1))
data = pd.concat([btc_lrr.rename('btc_lrr'), aapl_lrr.rename('aapl_lrr'), gas_lrr.rename('gas_lrr')], axis=1).dropna()

# Проверка гипотезы о равенстве средних - ANOVA тест
f_statistic, p_value = stats.f_oneway(data['btc_lrr'], data['aapl_lrr'], data['gas_lrr'])
alpha = 0.05 
if p_value < alpha:
    print("Отвергаем гипотезу о равенстве средних (мю), ответ - no")
else: # почему мю и сигма в ответах - непонятно...
    print("Не отвергаем гипотезу о равенстве средних (мю), ответ - yes")

# Проверка гипотезы о равенстве дисперсий - тест Бартлетта
statistic, p_value = stats.bartlett(data['btc_lrr'], data['aapl_lrr'], data['gas_lrr'])
alpha = 0.05  
if p_value < alpha:
    print("Отвергаем гипотезу о равенстве дисперсий (сигма), ответ - no")
else: # почему мю и сигма в ответах - непонятно...
    print("Не твергаем гипотезу о равенстве дисперсий (сигма), ответ - yes")

Не отвергаем гипотезу о равенстве средних (мю), ответ - yes
Отвергаем гипотезу о равенстве дисперсий (сигма), ответ - no


Записываем ответы

In [281]:
ANSWERS = {
    'tasks' : {
        'task1_mu' : {
            "answer" : 0.000663301751786781
        }, 
        'task1_sigma' : {
            "answer" : 0.038671397386446926
        }, 
        'task2_normal' : {
            'answer' : 'no'
        }, 
        'task3_mu' : {
            "answer" : 1.129638689602703e-17
        }, 
        'task3_sigma' : {
            "answer" : 1.0
        }, 
        'task4_90' : {
            "answer" : 0.00029177015646652567
        }, 
        'task4_90_int' : {
            "answer" : [-0.001194272684038336, 0.0017778129969713875]
        }, 
        'task4_95' : {
            "answer" : 0.00029177015646652567
        }, 
        'task4_95_int' : {
            "answer" : [-0.0016159487743707115, 0.002199489087303763]
        }, 
        'task4_99' : {
            "answer" : 0.00029177015646652567
        }, 
        'task4_99_int' : {
            "answer" : [-0.0024078070690674784, 0.00299134738200053]
        }, 
        'task5_corr' : {
            "answer" : "no"
        },
        'task6_corr_mu' : {
            "answer" : "yes"
        },
        'task6_corr_sigma' : {
            "answer" : "no"
        }
    }
}

In [282]:
import yaml
yaml_result = yaml.dump(ANSWERS)

print(yaml_result)

with open(f"hw03.yaml", "w") as f:
    f.write(yaml_result)

tasks:
  task1_mu:
    answer: 0.000663301751786781
  task1_sigma:
    answer: 0.038671397386446926
  task2_normal:
    answer: 'no'
  task3_mu:
    answer: 1.129638689602703e-17
  task3_sigma:
    answer: 1.0
  task4_90:
    answer: 0.00029177015646652567
  task4_90_int:
    answer:
    - -0.001194272684038336
    - 0.0017778129969713875
  task4_95:
    answer: 0.00029177015646652567
  task4_95_int:
    answer:
    - -0.0016159487743707115
    - 0.002199489087303763
  task4_99:
    answer: 0.00029177015646652567
  task4_99_int:
    answer:
    - -0.0024078070690674784
    - 0.00299134738200053
  task5_corr:
    answer: 'no'
  task6_corr_mu:
    answer: 'yes'
  task6_corr_sigma:
    answer: 'no'

